# 📊 Análisis espectral de la aceleración
Esta plantilla te ayudará a:
- Cargar una señal de aceleración vs tiempo
- Aplicar la Transformada Discreta de Fourier (DFT)
- Identificar los armónicos dominantes
- Reconstruir la señal con los primeros armónicos

👉 Asegúrate de tener tus datos en un archivo `.txt` o `.csv` con columnas de tiempo y aceleración.

In [ ]:
# ✅ Librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq

# Configuración de gráficos
plt.rcParams['figure.figsize'] = (12, 5)


## 📁 1. Cargar archivo de datos
Modifica esta celda para que apunte a tu archivo de datos con columnas de tiempo y aceleración.

In [ ]:
# Cargar datos desde archivo
# Reemplaza 'archivo.txt' con el nombre de tu archivo
df = pd.read_csv("archivo.txt", sep="\t", skiprows=2)
df.columns = ['Tiempo', 'Ángulo', 'Velocidad1', 'Aceleración1', 'Posición', 'Velocidad2', 'Aceleración2']

# Visualizar las primeras filas
df.head()

## 🔍 2. Seleccionar intervalo aproximadamente periódico
Selecciona una parte de la señal donde se vea un comportamiento cíclico.

In [ ]:
# Acotar datos a un intervalo periódico
inicio, fin = 2, 6  # segundos
mask = (df['Tiempo'] >= inicio) & (df['Tiempo'] <= fin)
t = df.loc[mask, 'Tiempo'].values
a = df.loc[mask, 'Aceleración1'].values

# Mostrar la señal
plt.plot(t, a)
plt.title("Aceleración vs Tiempo (segmento seleccionado)")
plt.xlabel("Tiempo (s)")
plt.ylabel("Aceleración (rad/s²)")
plt.grid()
plt.show()

## ⚙️ 3. Calcular la DFT y obtener el espectro de amplitudes

In [ ]:
# Parámetros de muestreo
N = len(t)
T = t[1] - t[0]
f_s = 1 / T

# FFT
yf = fft(a)
xf = fftfreq(N, T)[:N // 2]
amplitudes = 2.0 / N * np.abs(yf[:N // 2])

# Espectro
plt.stem(xf, amplitudes, use_line_collection=True)
plt.title("Espectro de Fourier")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 25)
plt.grid()
plt.show()

## 📌 4. Identificar los dos primeros armónicos dominantes

In [ ]:
# Filtrar frecuencias con amplitud significativa
threshold = np.max(amplitudes) * 0.2
dominant_indices = np.where(amplitudes > threshold)[0]
dominant_freqs = xf[dominant_indices]
dominant_amps = amplitudes[dominant_indices]

# Mostrar tabla
dominant_df = pd.DataFrame({
    "Frecuencia (Hz)": dominant_freqs,
    "Amplitud": dominant_amps
}).sort_values(by="Amplitud", ascending=False)

dominant_df.head(5)

## 🧱 5. Reconstrucción de la señal con los dos primeros armónicos

In [ ]:
# Seleccionar las dos frecuencias más fuertes
phases = np.angle(yf)
top_two = dominant_df.head(2)
reconstructed = np.zeros_like(t)

for f in top_two["Frecuencia (Hz)"]:
    idx = np.argmin(np.abs(xf - f))
    A = amplitudes[idx]
    phi = phases[idx]
    reconstructed += A * np.cos(2 * np.pi * f * t + phi)

# Comparación visual
plt.plot(t, a, label="Original")
plt.plot(t, reconstructed, label="Reconstruida (2 armónicos)", linestyle="--")
plt.title("Reconstrucción de la señal")
plt.xlabel("Tiempo (s)")
plt.ylabel("Aceleración (rad/s²)")
plt.legend()
plt.grid()
plt.show()

## ✏️ 6. Conclusión
Comenta lo siguiente:
- ¿Qué tan bien se aproxima la reconstrucción a la señal original?
- ¿Qué información se pierde?
- ¿Los armónicos encontrados tienen sentido físico?
- ¿Hay otros picos en el espectro? ¿Podrían deberse a ruido o efectos mecánicos del sistema?